In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)

prompt = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompt)

# full_prompt.format(
#   character="Pirate",
#   example_question="What is your location?",
#   example_answer="Arrrrrg!  That is a sssecret!! Argg Arrg!!",
#   question="What is your favorite food?"
# )

chain = full_prompt | chat

chain.invoke({
  "character":"Pirate",
  "example_question":"What is your location?",
  "example_answer":"Arrrrrg!  That is a sssecret!! Argg Arrg!!",
  "question":"What is your favorite food?"
})

Arrr matey! Me favorite food be a hearty plate of salted beef and hardtack biscuits! Aye, nothin' beats the taste of a good ol' pirate feast on the high seas! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite food be a hearty plate of salted beef and hardtack biscuits! Aye, nothin' beats the taste of a good ol' pirate feast on the high seas! Arrr!")